In [21]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [22]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches (current 3 hours),Snow Inches (current 3 hours)
0,0,Chokurdakh,RU,2020-01-17 00:41:48,70.63,147.92,-20.04,97,100,7.72,overcast clouds,0.0,0.0
1,1,Airai,TL,2020-01-17 00:41:49,-8.93,125.41,76.77,74,18,1.23,few clouds,0.0,0.0
2,2,Sitka,US,2020-01-17 00:41:49,57.05,-135.33,16.57,61,83,9.37,broken clouds,0.0,0.0
3,3,Port Alfred,ZA,2020-01-17 00:41:50,-33.59,26.89,71.01,95,0,16.44,clear sky,0.0,0.0
4,4,Oppdal,NO,2020-01-17 00:41:50,62.59,9.69,21.60,78,83,12.24,broken clouds,0.0,0.0


In [23]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [24]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [25]:
# Ask the customer to add preferences to filter the cities.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_pref = bool(input("Do you want it to be raining? (yes/no)"))
snow_pref = bool(input("Do you want it to be snowing? (yes/no)"))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no)yes
Do you want it to be snowing? (yes/no)no


In [26]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches (current 3 hours),Snow Inches (current 3 hours)
1,1,Airai,TL,2020-01-17 00:41:49,-8.93,125.41,76.77,74,18,1.23,few clouds,0.0,0.0
3,3,Port Alfred,ZA,2020-01-17 00:41:50,-33.59,26.89,71.01,95,0,16.44,clear sky,0.0,0.0
6,6,Santiago Del Estero,AR,2020-01-17 00:41:51,-27.80,-64.26,71.87,29,99,7.11,overcast clouds,0.0,0.0
8,8,Baculin,PH,2020-01-17 00:41:51,7.45,126.58,74.03,88,85,2.84,overcast clouds,0.0,0.0
11,11,Rikitea,PF,2020-01-17 00:41:52,-23.12,-134.97,77.49,78,11,16.28,few clouds,0.0,0.0
16,16,Marawi,PH,2020-01-17 00:41:55,8.00,124.29,81.00,78,46,1.99,scattered clouds,0.0,0.0
20,20,La Union,PE,2020-01-17 00:41:56,-5.40,-80.75,81.00,57,56,18.68,broken clouds,0.0,0.0
22,22,Busselton,AU,2020-01-17 00:41:57,-33.65,115.33,75.00,43,48,13.27,scattered clouds,0.0,0.0
24,24,Sinjai,ID,2020-01-17 00:41:58,-5.12,120.25,81.82,77,94,4.68,overcast clouds,0.0,0.0
27,27,East London,ZA,2020-01-17 00:41:59,-33.02,27.91,77.09,82,0,20.15,clear sky,0.0,0.0


In [27]:
# Create DataFrame called to store hotel names along with city, country, max temp, and coordinates.
add_hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
add_hotel_df["Hotel Name"] = ""
add_hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Airai,TL,76.77,-8.93,125.41,
3,Port Alfred,ZA,71.01,-33.59,26.89,
6,Santiago Del Estero,AR,71.87,-27.80,-64.26,
8,Baculin,PH,74.03,7.45,126.58,
11,Rikitea,PF,77.49,-23.12,-134.97,
16,Marawi,PH,81.00,8.00,124.29,
20,La Union,PE,81.00,-5.40,-80.75,
22,Busselton,AU,75.00,-33.65,115.33,
24,Sinjai,ID,81.82,-5.12,120.25,
27,East London,ZA,77.09,-33.02,27.91,


In [28]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [29]:
# Iterate through the DataFrame.
for index, row in add_hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        add_hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [32]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in add_hotel_df.iterrows()]

In [33]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = add_hotel_df[["Lat", "Lng"]]
max_temp = add_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))